# Extracting training data from the ODC <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_gm_2_annual](https://explorer.digitalearth.africa/ga_ls8c_gm_2_annual), [s2_l2a](https://explorer.digitalearth.africa/s2_l2a)


## Background

**Training data** is the most important part of any supervised machine learning workflow. The quality of the training data has a greater impact on the classification than the algorithm used. Large and accurate training data sets are preferable: increasing the training sample size results in increased classification accuracy ([Maxell et al 2018](https://www.tandfonline.com/doi/full/10.1080/01431161.2018.1433343)).  A review of training data methods in the context of Earth Observation is available [here](https://www.mdpi.com/2072-4292/12/6/1034) 

When creating training data, be sure to capture the **spectral variability** of the class, and to use imagery from the time period you want to classify (rather than relying on basemap composites). Another common problem with training data is **class imbalance**. This can occur when one of your classes is relatively rare and therefore the rare class will comprise a smaller proportion of the training set. When imbalanced data is used, it is common that the final classification will under-predict less abundant classes relative to their true proportion.

There are many platforms to use for gathering training data, the best one to use depends on your application. GIS platforms are great for collection training data as they are highly flexible and mature platforms; [Geo-Wiki](https://www.geo-wiki.org/) and [Collect Earth Online](https://collect.earth/home) are two open-source websites that may also be useful depending on the reference data strategy employed.

This notebook assumes the user has already collected reference data polygons, and now wishes to extract feature layers from the `open-data-cube` to assist in classifying satellite images.


## Description
This notebook will extract training data (feature layers) from the `open-data-cube` using geometries within a shapefile (or geojson). To do this, we rely on a custom `deafrica-sandbox-notebooks` function called `collect_training_data`, contained within the [deafrica_classificationtools](../Scripts/deafrica_classificationtools.py) script.  The goal of this notebook is to familarise users with this function so they can extract the appropriate data for their use-case.

1. Preview the polygons in our training data by plotting them on a basemap
2. Extract training data from the datacube using `collect_training_data`'s inbuilt feature layer parameters
3. Extract training data from the datacube using  a custom defined feature layer function that we can pass to `collect_training_data`
4. Export the training data to disk for use in subsequent scripts

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages


In [ ]:
# !pip install richdem

In [ ]:
%matplotlib inline

import sys
import os
import xarray as xr
import subprocess as sp
import numpy as np
import geopandas as gpd
import datacube
from datacube.utils.geometry import assign_crs
import warnings
warnings.filterwarnings("ignore")

sys.path.append('../Scripts')
from deafrica_plotting import map_shapefile
from deafrica_bandindices import calculate_indices
from deafrica_temporal_statistics import temporal_statistics
from deafrica_classificationtools import collect_training_data 

from xr_geomedian_tmad import xr_geomedian_tmad

%load_ext autoreload
%autoreload 2

## Analysis parameters

* `path`: The path to the input shapefile from which we will extract training data. A default shapefile is provided.
* `field`: This is the name of column in your shapefile attribute table that contains the class labels. **The class labels must be integers**
* `ncpus`: Set this value to > 1 to parallize the collection of training data. eg. npus=8. 

> **Note**: With supervised classification, its common to have many, many labelled geometries in the training data. `collect_training_data` can parallelize across the geometries in order to speed up the extracting of training data. Setting `ncpus>1` will automatically trigger the parallelization, however, its best to set `ncpus=1` to begin with to assist with debugging before triggering the parallelization. 


In [ ]:
path = 'data/training/test_training_dataset.shp' 
field = 'class'

# automatically detect number of cpus, adjust to [-3:] if working on deafault Sandbox
ncpus= int(float(sp.getoutput('env | grep CPU')[-4:]))

print('ncpus = '+str(ncpus))

## Preview input data

We can load and preview our input data shapefile using `geopandas`. The shapefile should contain a column with class labels (e.g. 'class'). These labels will be used to train our model. 

> Remember, the class labels **must** be represented by `integers`.


In [ ]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

In [ ]:
# Plot training data in an interactive map
map_shapefile(input_data, attribute=field)

## Extracting training data

The function `collect_training_data` takes our shapefile containing class labels and extracts training data from the datacube over the locations specified by the input geometries. The function will also pre-process our training data by stacking the arrays into a useful format and removing an `NaN` (not-a-number) values. 

`Collect_training_data` has the ability to generate many different types of **feature layers**, relatively simple layers can be calculated using pre-defined parameters within the function; more complex layers can be computed by passing in a `custom_func`. To begin with, let's try generating feature layers using the pre-defined methods.

The in-built feature layer parameters are described below:
* `product`: The name of the product to extract from the datacube. In this example we use a geomedian composite from 2018, `'ga_ls8c_gm_2_annual'`
* `time`: The time range from which to extract data
* `calc_indices`: This parameter provides a method for calculating a number of remote sensing indices (e.g. `['NDWI', 'NDVI']`). Any of the indices found in the [deafrica_bandindices](../Scripts/deafrica_bandindices.py) script can be used here
* `drop`: If this variable is set to True, and 'calc_indices' are supplied, the spectral bands will be dropped from the dataset leaving only the band indices as data variables in the dataset.
* `reduce_func`: The classification models we're applying here require our training data to be in two dimensions (ie. `x` & `y`). If our data has a time-dimension (e.g. if we load in an annual time-series of satellite images) then we need to collapse the time dimension.  "reduce_func" is simply the summary statistic used to collapse the temporal dimension. Options are 'mean', 'median', 'std', 'max', 'min', 'geomedian'. In the default example we are loading a geomedian composite, so there is no time dimension to reduce.
* `zonal_stats`: An optional string giving the names of zonal statistics to calculate across each polygon. Default is None (all pixel values are returned). Supported values are 'mean', 'median', 'max', 'min', and 'std'.

In addition to the parameters required for `collect_training_data`, we also need to set up a few parameters for the open-data-cube query, such as `measurements` (the bands to load from the satellite), the `resolution` (the cell size), and the `output_crs` (the output projection). 

In [ ]:
#set up our inputs to collect_training_data
products =  ['ga_ls8c_gm_2_annual']
time = ('2018')
reduce_func = None
calc_indices = ['NDVI','LAI', 'NDWI']
drop = False
zonal_stats = 'median' 

# Set up the inputs for the ODC query
measurements =  ['blue','green','red','nir','swir_1','swir_2']
resolution = (-30,30)
output_crs='epsg:6933'

In [ ]:
# generate a datacube query object
query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs': output_crs,
    'group_by' : 'solar_day',
}

Now let's run the `collect_training_data` function.

In [ ]:
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    products=products,
                                    dc_query=query,
                                    ncpus=ncpus,
                                    field=field,
                                    calc_indices=calc_indices,
                                    reduce_func=reduce_func,
                                    drop=drop,
                                    zonal_stats=zonal_stats)

The function returns two numpy arrays, the first (`column_names`) contains a list of the names of the feature layers we've computed:

In [ ]:
print(column_names)

The second array (`model_input`) contains the data from our labelled geometries. The first item in the array is the class integer (e.g. in the default example 1. 'crop', or 0. 'noncrop'), the second set of items are the values for each feature layer we computed:

In [ ]:
print(np.array_str(model_input, precision=2, suppress_small=True))

## Optional: Custom feature layers

The feature layers that are most relevant for discriminating the classes of your classification problem may be more complicated than those provided in the `collect_training_data` function.  In this case, we can pass a custom feature layer function through the `custom_func` parameter.   

* `custom_func`: A custom function for generating feature layers. If this parameter is set, all other options (excluding 'zonal_stats'), will be ignored. The result of the 'custom_func' must be a single xarray dataset containing 2D coordinates (i.e x, y; no time dimension). The custom function has access to the datacube dataset extracted using the `dc_query` params. To load other datasets, you can use the `like=ds.geobox` parameter in `dc.load`

First, lets define a custom feature layer function that will demonstrate how to extract more complicated temporal statistics.

In [ ]:
def two_seasons_gm_mads(ds):
    dc = datacube.Datacube(app='training')
    
    ds1 = ds.sel(time=slice('2019-01', '2019-06'))
    ds2 = ds.sel(time=slice('2019-07', '2019-12')) 
    
    def fun(ds, era):
        
        gm_mads = xr_geomedian_tmad(ds)
        gm_mads = calculate_indices(gm_mads,
                               index=['NDVI', 'LAI'],
                               drop=False,
                               collection='s2')
        
        gm_mads['edev'] = -np.log(gm_mads['edev'])
        gm_mads['sdev'] = -np.log(gm_mads['sdev'])
        gm_mads['bcdev'] = -np.log(gm_mads['bcdev'])

        for band in gm_mads.data_vars:
            gm_mads = gm_mads.rename({band:band+era})
        
        return gm_mads
    
    epoch1_gm_mads = fun(ds1, era='_S1')
    epoch2_gm_mads = fun(ds2, era='_S2')
    
    result = xr.merge([epoch1_gm_mads,
                       epoch2_gm_mads],
                      compat='override')
    
    print(result)
    return result.squeeze()

Now, we can pass this function to `collect_training_data`.  For each of the geometries in our shapefile we will extract temporal statistics, and elevation data.

Because we are now interested in calculating a range of temporal statistics, we will redefine our intial parameters to include a time-series of Sentinel-2 data (whereas above we loaded data from a geomedian composite).  Remember, passing in a `custom_func` to `collect_training_data` means many of the other feature layer parameters are ignored.

In [ ]:
#set up our inputs to collect_training_data
products =  ['s2_l2a']
time = ('2019-01','2019-12')
zonal_stats = 'median' 

# Set up the inputs for the ODC query
measurements =  ['red', 'nir', 'blue','green', 'swir_1', 'swir_2']
resolution = (-20,20)
output_crs='epsg:6933'

In [ ]:
#generate a new datacube query object
query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs': output_crs,
    'group_by' : 'solar_day',
}

As we are now extracting 12-months of Sentinel-2 data from the datacube for every geometry, the load times will be much slower.

In [ ]:
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    products=products,
                                    dc_query=query,
                                    ncpus=ncpus,
                                    field=field,
                                    zonal_stats=zonal_stats,
                                    custom_func=two_seasons_gm_mads
                                    )

In [ ]:
print(column_names)
print('')
print(np.array_str(model_input, precision=2, suppress_small=True))

## Export training data

Once we've collected all the training data we require, we can write the data to disk. This will allow us to import the data in the next step(s) of the workflow.


In [ ]:
#set the name and location of the output file
output_file = "results/training_data/test_training_data.txt"

In [ ]:
#Export files to disk
np.savetxt(output_file, model_input, header=" ".join(column_names), fmt="%4f")

## Recommended next steps

To continue working through the notebooks in this `Scalable Machine Learning on the ODC` workflow, go to the next notebook `2_Inspect_training_data.ipynb`.

1. **Extracting_training_data (this notebook)**
2. [Inspect_training_data](2_Inspect_training_data.ipynb)
3. [Train_fit_evaluate_classifier](3_Train_fit_evaluate_classifier.ipynb)
4. [Predict](4_Predict.ipynb)
5. [Accuracy_assessment](5_Accuracy_assessment.ipynb)
6. [Object-based_filtering](6_Object-based_filtering_(optional).ipynb)


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** August 2020

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)